In [2]:
'''Question 1

Run it to test that it's working locally:

docker run -it --rm -p 9696:9696 zoomcamp-model:v001

And in another terminal, execute q6_test.py file:

python q6_test.py

You should see this:

{'get_card': True, 'get_card_probability': <value>}

Here <value> is the probability of getting a credit card. You need to choose the right one.

    0.289
    0.502
    0.769
    0.972
 '''
import requests


url = "http://localhost:9696/predict"

client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
response = requests.post(url, json=client).json()

print(response)
# A: 0.7692649226628628

ConnectionError: HTTPConnectionPool(host='localhost', port=9696): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8c9f807bb0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [3]:
'''
Question 2

What's the version of kind that you have?

Use kind --version to find out.
Creating a cluster

Now let's create a cluster with kind:

kind create cluster

And check with kubectl that it was successfully created:

kubectl cluster-info
'''
!kind --version
#!kind create cluster
!kubectl cluster-info --context kind-kind
# A: 0.17.0

kind version 0.17.0
Kubernetes control plane is running at https://127.0.0.1:64800
CoreDNS is running at https://127.0.0.1:64800/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [4]:
!kubectl cluster-info --context kind-kind

Kubernetes control plane is running at https://127.0.0.1:64800
CoreDNS is running at https://127.0.0.1:64800/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [5]:
'''
Question 3

What's the smallest deployable computing unit that we can create and manage in Kubernetes (kind in our case)?

    Node
    Pod
    Deployment
    Service
'''

"\nQuestion 3\n\nWhat's the smallest deployable computing unit that we can create and manage in Kubernetes (kind in our case)?\n\n    Node\n    Pod\n    Deployment\n    Service\n"

In [6]:
'''Question 4

Now let's test if everything works. Use kubectl to get the list of running services.

What's the Type of the service that is already running there?

    ClusterIP
    NodePort
    LoadBalancer
    ExternalName
'''
!kubectl get services
#A: ClusterIP

NAME                  TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
credit-card-service   LoadBalancer   10.96.102.229   <pending>     80:31339/TCP   19h
kubernetes            ClusterIP      10.96.0.1       <none>        443/TCP        20h


In [7]:
'''
Question 5

To be able to use the docker image we previously created (zoomcamp-model:v001), we need to register it with kind.

What's the command we need to run for that?

    kind create cluster
    kind build node-image
    kind load docker-image
    kubectl apply
'''
!kind load docker-image zoomcamp-model:v002

Image: "zoomcamp-model:v002" with ID "sha256:951a2ef903b849d0ed069942d05aa2edd5e50cb7f91ceb9a7301068d0910a18c" found to be already present on all nodes.


In [8]:
'''
Question 6

Now let's create a deployment config (e.g. deployment.yaml):


Replace <Image>, <Memory>, <CPU>, <Port> with the correct values.

What is the value for <Port>?

Apply this deployment using the appropriate command and get a list of running Pods. You can see one running Pod.'''

# A: 9696
!kubectl apply -f deployment.yaml
!kubectl get deployment
!kubectl get pod

deployment.apps/credit-card unchanged
NAME          READY   UP-TO-DATE   AVAILABLE   AGE
credit-card   1/1     1            1           19h
NAME                           READY   STATUS    RESTARTS   AGE
credit-card-54c4c78975-fhzxp   1/1     Running   0          19h


In [9]:
'''
Question 7

Let's create a service for this deployment (service.yaml):
Fill it in. What do we need to write instead of <???>?

Apply this config file.
'''
# A: credit-card
!kubectl apply -f service.yaml
!kubectl get service

service/credit-card-service unchanged
NAME                  TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
credit-card-service   LoadBalancer   10.96.102.229   <pending>     80:31339/TCP   19h
kubernetes            ClusterIP      10.96.0.1       <none>        443/TCP        20h


In [10]:
'''
Testing the service

We can test our service locally by forwarding the port 9696 on our computer to the port 80 on the service:

kubectl port-forward service/<Service name> 9696:80

Run q6_test.py (from the homework 5) once again to verify that everything is working. You should get the same result as in Question 1.
'''
# This needs to run in a terminal permanently for it to work
#!kubectl port-forward service/credit-card-service 9696:80

Forwarding from 127.0.0.1:9696 -> 9696
Forwarding from [::1]:9696 -> 9696
^C


In [13]:
import requests


url = "http://localhost:9696/predict"

client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
response = requests.post(url, json=client).json()

print(response)

{'get_card': True, 'get_card_probability': 0.7692649226628628}


In [14]:
'''
Autoscaling

Now we're going to use a HorizontalPodAutoscaler (HPA for short) that automatically updates a workload resource (such as our deployment), with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

kubectl autoscale deployment credit-card --name credit-card-hpa --cpu-percent=20 --min=1 --max=3

You can check the current status of the new HPA by running:

kubectl get hpa

The output should be similar to the next:

NAME              REFERENCE                TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
credit-card-hpa   Deployment/credit-card   1%/20%    1         3         1          27s

TARGET column shows the average CPU consumption across all the Pods controlled by the corresponding deployment. Current CPU consumption is about 0% as there are no clients sending requests to the server.

    Note: In case the HPA instance doesn't run properly, try to install the latest Metrics Server release from the components.yaml manifest:

    kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml
    '''
!kubectl autoscale deployment credit-card --name credit-card-hpa --cpu-percent=20 --min=1 --max=3
!kubectl get hpa # on MACOS for live feedback: watch kubectl get hpa

horizontalpodautoscaler.autoscaling/credit-card-hpa autoscaled
NAME              REFERENCE                TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
credit-card-hpa   Deployment/credit-card   <unknown>/20%   1         3         0          0s


In [ ]:
# Load test
from time import sleep
import requests

url = "http://localhost:9696/predict"

client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}

while True:
    sleep(0.01)
    response = requests.post(url, json=client).json()
    #print(response)

In [ ]:
'''
Question 8 (optional)

Run kubectl get hpa credit-card-hpa --watch command to monitor how the autoscaler performs. Within a minute or so, you should see the higher CPU load; and then - more replicas. What was the maximum amount of the replicas during this test?

    1
    2
    3
    4

    Note: It may take a few minutes to stabilize the number of replicas. Since the amount of load is not controlled in any way it may happen that the final number of replicas will differ from initial.
'''
